In [54]:
# 环境配置
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np   
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score   
from sklearn.metrics import accuracy_score

# 表格1的编码量化和缺失值处理

In [55]:
# 读取表格1数据
path = '..\Question\附件.xlsx'
excel1 = pd.read_excel(path,sheet_name = 0) 

In [56]:
excel1.head()

,文物编号,纹饰,类型,颜色,表面风化
0,1,C,高钾,蓝绿,无风化
1,2,A,铅钡,浅蓝,风化
2,3,A,高钾,蓝绿,无风化
3,4,A,高钾,蓝绿,无风化
4,5,A,高钾,蓝绿,无风化


In [57]:
excel1.columns

Index(['文物编号', '纹饰', '类型', '颜色', '表面风化'], dtype='object')

In [58]:
# 对各分类变量Label Encoding编码量化
le = LabelEncoder()
excel1_encoding = excel1.copy(deep=True)
excel1_columns_lst = list(excel1.columns)
excel1_columns_lst.remove('文物编号')
for column in excel1_columns_lst:
    excel1_encoding[column] = le.fit_transform(excel1_encoding[column])
    print(le.classes_)

['A' 'B' 'C']
['铅钡' '高钾']
['浅绿' '浅蓝' '深绿' '深蓝' '紫' '绿' '蓝绿' '黑' nan]
['无风化' '风化']


In [59]:
# 将颜色中种类为8替换为NaN
excel1_encoding.loc[excel1_encoding['颜色'] == 8, '颜色'] = np.nan

In [60]:
excel1_encoding.loc[18,list(excel1_encoding.columns)]

文物编号    19.0
纹饰       0.0
类型       0.0
颜色       NaN
表面风化     1.0
Name: 18, dtype: float64

In [61]:
# KNN插补的特征列  
X = excel1_encoding[['纹饰', '类型', '表面风化']]
y = excel1_encoding['颜色']
# 分离出有缺失和没有缺失的行 
non_missing = ~excel1_encoding['颜色'].isna()  
X_train, y_train = X[non_missing], y[non_missing]  

# 初始化KNN回归器  KNeighborsRegressor连续，KNeighborsClassifier离散
knn = KNeighborsClassifier(n_neighbors=1)  
knn.fit(X_train, y_train)  

# 预测缺失的颜色  
missing_indices = excel1_encoding['颜色'].isna()  
X_missing = X[missing_indices]
predicted_color = knn.predict(X_missing)  
  
# 将预测值放回DataFrame  
excel1_encoding.loc[missing_indices, '颜色'] = predicted_color 

In [62]:
excel1_encoding.loc[18,list(excel1_encoding.columns)]

文物编号    19.0
纹饰       0.0
类型       0.0
颜色       1.0
表面风化     1.0
Name: 18, dtype: float64

In [63]:
excel1_encoding.to_excel('excel1_encoding.xlsx',index=False)

# 表格2的缺失值处理

In [64]:
# 读取数据
excel2 = pd.read_excel(path,sheet_name=1)

In [65]:
excel2.head()

,文物采样点,文物编号,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
0,01,1,69.33,NaN,9.99,6.32,0.87,3.93,1.74,3.87,NaN,NaN,1.17,NaN,NaN,0.39
1,02,2,36.28,NaN,1.05,2.34,1.18,5.73,1.86,0.26,47.43,NaN,3.57,0.19,NaN,NaN
2,03部位1,3,87.05,NaN,5.19,2.01,NaN,4.06,NaN,0.78,0.25,NaN,0.66,NaN,NaN,NaN
3,03部位2,3,61.71,NaN,12.37,5.87,1.11,5.50,2.16,5.09,1.41,2.86,0.70,0.10,NaN,NaN
4,04,4,65.88,NaN,9.67,7.12,1.56,6.44,2.06,2.18,NaN,NaN,0.79,NaN,NaN,0.36


In [66]:
# 以0填补空缺值
excel2_filled = excel2.fillna(0) 

In [67]:
excel2_filled.head()

,文物采样点,文物编号,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
0,01,1,69.33,0.0,9.99,6.32,0.87,3.93,1.74,3.87,0.00,0.00,1.17,0.00,0.0,0.39
1,02,2,36.28,0.0,1.05,2.34,1.18,5.73,1.86,0.26,47.43,0.00,3.57,0.19,0.0,0.00
2,03部位1,3,87.05,0.0,5.19,2.01,0.00,4.06,0.00,0.78,0.25,0.00,0.66,0.00,0.0,0.00
3,03部位2,3,61.71,0.0,12.37,5.87,1.11,5.50,2.16,5.09,1.41,2.86,0.70,0.10,0.0,0.00
4,04,4,65.88,0.0,9.67,7.12,1.56,6.44,2.06,2.18,0.00,0.00,0.79,0.00,0.0,0.36


In [68]:
# 计算成分比例累加和
excel2_filled['成分比例累加和'] = excel2_filled.iloc[:,2:].sum(axis=1)

In [69]:
excel2_filled.iloc[15:20,:]

,文物采样点,文物编号,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),成分比例累加和
15,13,13,59.01,2.86,12.53,8.70,0.00,6.16,2.88,4.73,0.00,0.0,1.27,0.00,0.0,0.0,98.14
16,14,14,62.47,3.38,12.28,8.23,0.66,9.23,0.50,0.47,1.62,0.0,0.16,0.00,0.0,0.0,99.00
17,15,15,61.87,3.21,7.44,0.00,1.02,3.15,1.04,1.29,0.19,0.0,0.26,0.00,0.0,0.0,79.47
18,16,16,65.18,2.10,14.52,8.27,0.52,6.18,0.42,1.07,0.11,0.0,0.00,0.04,0.0,0.0,98.41
19,17,17,60.71,2.12,5.71,0.00,0.85,0.00,1.04,1.09,0.19,0.0,0.18,0.00,0.0,0.0,71.89


In [70]:
# 85%~105%之间的数据视为有效数据，剔除异常数据
com_sum = excel2_filled['成分比例累加和']
outliers_data = excel2_filled[(com_sum < 85) | (com_sum > 105)]
outliers_data

,文物采样点,文物编号,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),成分比例累加和
17,15,15,61.87,3.21,7.44,0.0,1.02,3.15,1.04,1.29,0.19,0.0,0.26,0.0,0.0,0.0,79.47
19,17,17,60.71,2.12,5.71,0.0,0.85,0.00,1.04,1.09,0.19,0.0,0.18,0.0,0.0,0.0,71.89


In [71]:
excel2_filled.drop(outliers_data.index,inplace=True)

In [72]:
excel2_filled.iloc[15:20,:]

,文物采样点,文物编号,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),成分比例累加和
15,13,13,59.01,2.86,12.53,8.70,0.00,6.16,2.88,4.73,0.00,0.00,1.27,0.00,0.00,0.0,98.14
16,14,14,62.47,3.38,12.28,8.23,0.66,9.23,0.50,0.47,1.62,0.00,0.16,0.00,0.00,0.0,99.00
18,16,16,65.18,2.10,14.52,8.27,0.52,6.18,0.42,1.07,0.11,0.00,0.00,0.04,0.00,0.0,98.41
20,18,18,79.46,0.00,9.42,0.00,1.53,3.05,0.00,0.00,0.00,0.00,1.36,0.07,2.36,0.0,97.25
21,19,19,29.64,0.00,0.00,2.93,0.59,3.57,1.33,3.51,42.82,5.35,8.83,0.19,0.00,0.0,98.76


In [73]:
excel2_filled.head()

,文物采样点,文物编号,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),成分比例累加和
0,01,1,69.33,0.0,9.99,6.32,0.87,3.93,1.74,3.87,0.00,0.00,1.17,0.00,0.0,0.39,97.61
1,02,2,36.28,0.0,1.05,2.34,1.18,5.73,1.86,0.26,47.43,0.00,3.57,0.19,0.0,0.00,99.89
2,03部位1,3,87.05,0.0,5.19,2.01,0.00,4.06,0.00,0.78,0.25,0.00,0.66,0.00,0.0,0.00,100.00
3,03部位2,3,61.71,0.0,12.37,5.87,1.11,5.50,2.16,5.09,1.41,2.86,0.70,0.10,0.0,0.00,98.88
4,04,4,65.88,0.0,9.67,7.12,1.56,6.44,2.06,2.18,0.00,0.00,0.79,0.00,0.0,0.36,96.06


# 数据集成<br>
将表格1中的表面风化融合进入表格2中，根据文物编号

In [74]:
merged_excel = pd.merge(excel2_filled, excel1_encoding[['文物编号', '表面风化',"类型"]], on='文物编号', how='left') 

In [81]:
merged_excel.iloc[1,16]

99.88999999999999

### 对数据进行标准化处理

In [82]:
standard_excel = merged_excel.copy()
# 将数据中的所有化学成分含量都乘以标准化因子
for key, item in standard_excel.iloc[:,16].items():
    # 标准化因子
    standard = 100 / standard_excel.iloc[key,16]
    # 标准化
    standard_excel.iloc[key,2:16] = standard_excel.iloc[key,2:16] * standard
    standard_excel.iloc[key,16] = standard_excel.iloc[key,2:16].sum()

TypeError: _LocationIndexer.__call__() takes from 1 to 2 positional arguments but 3 were given

In [77]:
standard_excel

,文物采样点,文物编号,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),成分比例累加和,表面风化,类型
0,01,1,67.673013,0.000000,9.751239,6.168952,0.849207,3.836073,1.698414,3.777507,0.000000,0.000000,1.142037,0.000000,0.0,0.380679,95.277121,0,1
1,02,2,36.240092,0.000000,1.048845,2.337426,1.178702,5.723697,1.857954,0.259714,47.377827,0.000000,3.566073,0.189791,0.0,0.000000,99.780121,1,0
2,03部位1,3,87.050000,0.000000,5.190000,2.010000,0.000000,4.060000,0.000000,0.780000,0.250000,0.000000,0.660000,0.000000,0.0,0.000000,100.000000,0,1
3,03部位2,3,61.018848,0.000000,12.231456,5.804256,1.097568,5.438400,2.135808,5.032992,1.394208,2.827968,0.692160,0.098880,0.0,0.000000,97.772544,0,1
4,04,4,63.284328,0.000000,9.289002,6.839472,1.498536,6.186264,1.978836,2.094108,0.000000,0.000000,0.758874,0.000000,0.0,0.345816,92.275236,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,54严重风化点,54,16.583012,0.000000,0.000000,0.000000,1.075812,3.537580,0.000000,1.298728,56.659432,0.000000,13.694796,1.085504,0.0,0.000000,93.934864,1,0
63,55,55,47.235838,2.611898,0.000000,1.089094,0.000000,1.397510,0.000000,0.828868,31.728296,7.662210,0.337330,0.000000,0.0,0.000000,92.891044,0,0
64,56,56,26.887960,0.000000,0.000000,1.116104,0.000000,1.706440,0.000000,0.728696,38.049000,14.251080,2.342896,0.000000,0.0,0.000000,85.082176,1,0
65,57,57,23.505874,0.000000,0.000000,1.211357,0.000000,2.015846,0.000000,1.072652,41.703970,15.997310,0.000000,0.000000,0.0,0.000000,85.507009,1,0


In [78]:
with pd.ExcelWriter("merged_excel.xlsx") as writer:
    merged_excel.to_excel(writer, sheet_name="origin")
    standard_excel.to_excel(writer, sheet_name="standard")

PermissionError: [Errno 13] Permission denied: 'merged_excel.xlsx'